# TRABALHO DE REDES NEURAIS ARTIFICIAIS PROFUNDAS

## Indice

1. [Objetivo](#1)
2. [Abordagens comuns e trabalhos correlatos](#2)
3. [Proposta implementada](#3)
4. [Resultados alcançados](#4)
5. [Discussão final](#5)
6. [Referências](#6)

## Objetivo <a class="anchor" id="1"></a>

Temos como meta, conseguir segmementar terrenos, visando ter noção de propriedades rurais podendo ter noção de valor, atraves de images de satalete, em 5 grupos, que são construções, campos, florestas, estradas e em extenções de agua.

## Abordagens comuns e trabalhos correlatos <a class="anchor" id="2"></a>



## Proposta implementada e Resultados alcançados<a class="anchor" id="3"></a>

Das tecnicas estudadas, optamos pela [YOLO8](https://yolov8.com/) que em seu site é definica como "Um novo estado da arte em visão computacional, suportando tarefas de detecção, classificação e segmentação de objetos". Tendo em mente que o mesmo possui uma API de facil configuração e ser open source sendo isso fatores decisivos para a escolha.
Para os dados, usamos uma base disponivel no site do [Roboflow](https://universe.roboflow.com/myproject-gccot/landscape-segmentation/dataset/5/download). Esse dataset possui 2696 imagens para treinamento e 289 para validação.
Para isso decidimos fazer 3 treinamentos, que serão apresentados a seguir.



In [1]:
from ultralytics import YOLO, settings

settings.update({'runs_dir': './runs'})

model = YOLO('yolov8n-seg')  # yolov8n_custom.yaml

### Treinamento 1

Foi criada uma rede com o treinamento para 30 epocas, sem nenhuma opçao de otmizacao.

In [ ]:
model.train(data='data.yaml',
            epochs=30,
            imgsz=640
            )

![Saida do treinamento da primeira rede.](img/img_a_01.png)

Como podemos ver a saida do treinamento da primeira rede, levou mais de 5 horas.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da primeira rede.](img/img_a_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da primeira rede.](img/img_a_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da primeira rede.](img/img_a_04.jpg)

![Saida do treinamento da primeira rede.](img/img_a_05.jpg)

![Saida do treinamento da primeira rede.](img/img_a_06.jpg)

### Treinamento 2

Para o caso 2 foi criada uma rede com o treinamento para 100 epocas, sem nenhuma opçao de otmizacao.

In [ ]:

model.train(data='data.yaml',
            epochs=100,
            imgsz=640
            )


![Saida do treinamento da segunda rede.](img/img_b_01.png)

Como podemos ver a saida do treinamento da segunda rede, levou mais de 17 horas.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da segunda rede.](img/img_b_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da segunda rede.](img/img_b_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da segunda rede.](img/img_b_04.jpg)

![Saida do treinamento da segunda rede.](img/img_b_05.jpg)

![Saida do treinamento da segunda rede.](img/img_b_06.jpg)

### Treinamento 3

E para o ultimo caso utilizou-se uma rede com 30 epocas, com fatores de otimizacao, sendo o de "patience" o que indica o fim quando existe pouco avanço dentro do treinamento de epocas.

In [ ]:

model.train(data='data.yaml',
            epochs=30,
            patience=8,
            batch=16,  # number of images per batch (-1 for AutoBatch)
            imgsz=640,
            workers=8,
            pretrained=True,
            resume=False,  # resume training from last checkpoint
            single_cls=False,  # Whether all classes will be the same (just one class)
            # project='runs/detect',  # Default = /home/{user}/Documents/ultralytics/runs
            box=7.5,  # More recall, better IoU, less precission, 
            cls=0.5,  # Bbox class better
            dfl=1.5,  # Distribution Focal Loss. Better bbox boundaries
            val=True,
            # Augmentations
            degrees=0.3,
            hsv_s=0.3,
            hsv_v=0.3,
            scale=0.5,
            fliplr=0.5
            )


![Saida do treinamento da terceira rede.](img/img_c_01.png)

Como podemos ver a saida do treinamento da terceira rede, levou mais de 1 hora e 30 minutos.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da terceira rede.](img/img_c_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da terceira rede.](img/img_c_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da terceira rede.](img/img_c_04.jpg)

![Saida do treinamento da terceira rede.](img/img_c_05.jpg)

![Saida do treinamento da terceira rede.](img/img_c_06.jpg)

## Discussão final <a class="anchor" id="5"></a>

Vimos que todos os modelos convergiram, 


![Imagem obtida pelo modelo 2 com google maps.](img/img_d_01.png)

In [3]:
from PIL import ImageGrab
import cv2
import numpy as np
from collections import defaultdict

def real_time_maps(model):

    track_history = defaultdict(lambda: [])

    while True:
        img = ImageGrab.grab(bbox=(100,100,1000,1000))
    
        results = model.track(img, persist=True)
    
        # Process results list
        for result in results:
            # Visualize the results on the frame
            img = result.plot()
    
            if True:
                try:
                    # Get the boxes and track IDs
                    boxes = result.boxes.xywh.cpu()
                    track_ids = result.boxes.id.int().cpu().tolist()
    
                    # Plot the tracks
                    for box, track_id in zip(boxes, track_ids):
                        x, y, w, h = box
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  # x, y center point
                        if len(track) > 30:  # retain 90 tracks for 90 frames
                            track.pop(0)
    
                        # Draw the tracking lines
                        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                        cv2.polylines(img, [points], isClosed=False, color=(230, 0, 0), thickness=5)
                except:
                    pass
                
        cv2.imshow('Imagem do GoogleMaps', img)
    
        k = cv2.waitKey(1)
        if k == ord('q'):
            break
        
    cv2.destroyAllWindows()

In [4]:
model = YOLO('runs/segment/train5/weights/best.pt')

real_time_maps(model)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 640x640 (no detections), 288.7ms
Speed: 13.8ms preprocess, 288.7ms inference, 1188.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 208.5ms
Speed: 6.7ms preprocess, 208.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Aguas, 202.2ms
Speed: 4.2ms preprocess, 202.2ms inference, 12.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Aguas, 202.4ms
Speed: 8.4ms preprocess, 202.4ms inference, 7.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Campo, 1 Floresta, 3 Aguas, 244.7ms
Speed: 8.1ms prepr

In [5]:
model = YOLO('runs/segment/train52/weights/best.pt')

real_time_maps(model)


0: 640x640 (no detections), 135.4ms
Speed: 6.3ms preprocess, 135.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 167.6ms
Speed: 5.1ms preprocess, 167.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 147.8ms
Speed: 2.4ms preprocess, 147.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 152.3ms
Speed: 4.8ms preprocess, 152.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 143.7ms
Speed: 4.5ms preprocess, 143.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 144.1ms
Speed: 5.5ms preprocess, 144.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 151.4ms
Speed: 5.0ms preprocess, 151.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 139.7ms
Speed: 5.2ms prepr

In [6]:
model = YOLO('runs/segment/train522/weights/best.pt')

real_time_maps(model)


0: 640x640 (no detections), 157.0ms
Speed: 3.2ms preprocess, 157.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 164.2ms
Speed: 4.2ms preprocess, 164.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 127.5ms
Speed: 2.8ms preprocess, 127.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 121.6ms
Speed: 4.7ms preprocess, 121.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 148.2ms
Speed: 5.0ms preprocess, 148.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 153.4ms
Speed: 5.3ms preprocess, 153.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 150.1ms
Speed: 4.5ms preprocess, 150.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 151.6ms
Speed: 5.2ms prepr

## Referências  <a class="anchor" id="6"></a>

* https://universe.roboflow.com/myproject-gccot/landscape-segmentation/dataset/5/download
* https://github.com/WellingtonDev25/yolov8_amostra/blob/main/YoloV8_Amostra.ipynb
* https://csvisaocomputacional.com.br/cursoyolo
* https://docs.ultralytics.com/modes/track/#multithreaded-tracking
* https://www.kaggle.com/datasets/sshikamaru/car-object-detection/code